In [17]:
import pandas as pd
import numpy  as np
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [5]:
train_data = pd.read_parquet('../train_v1.parquet')
test_data  = pd.read_parquet('../test_v1.parquet')
# sample     = pd.read_parquet('../')
# solution   = pd.read_parquet('../input/house-prices-advanced-regression-solution-file/solution.csv')read_parquet

In [15]:
#===========================================================================
# select some features
#===========================================================================
features = set(list(train_data.columns))
features.remove("CI_HOUR")
features = list(features)
# print(len(set(list(train_data.columns))), len(features))

In [21]:
X = train_data[features]
y = train_data["CI_HOUR"]
X_test = test_data[features]
# y_ture = test_data["CI_HOUR"]
# train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=1004)

In [22]:
X      = X.to_numpy()
y      = y.to_numpy().reshape(-1, 1)
X_test = X_test.to_numpy()
# y_ture = y_ture.to_numpy().reshape(-1, 1)

In [23]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
predictions_array =[]
CV_score_array    =[]
for train_index, test_index in kf.split(X):
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    regressor = TabNetRegressor(verbose=0,seed=1004)
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=300, max_epochs=2000,
              eval_metric=['rmse'])
    CV_score_array.append(regressor.best_cost)
    predictions_array.append(np.expm1(regressor.predict(X_test)))

predictions = np.mean(predictions_array,axis=0)